Preprocessing of data and sentiment analysis

In [2]:
#LIBRARIES
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [3]:
#LOAD JSON FILES
path1 = 'playlist_data.json'
path2 = 'track_data.json'
path3 = 'track_map.json'
path4 = 'playlist_summaries.json'
path5 = 'track_audio_data.json'

playlist_json = json.load(open(path1, 'r'))
track_data_json = json.load(open(path2, 'r'))
meta_json = json.load(open(path5, 'r'))

In [4]:
#DELETE JSON OBJECTS WITH ALL "NONE ENTRIES" IN THE METADATA
filtered = {k: v for k, v in meta_json.items() if v is not None}
meta_json.clear()
meta_json.update(filtered)

In [5]:
def merge(dict1, dict2):
    #MERGE TWO DICTIONARIES
    res = {**dict1, **dict2}
    return res

In [6]:

#MERGE TWO DICTIONARIES AND ADD EMPTY LIST AS ENTRY OF PLAYLIST TO EACH OBJECT
final = {}
for i in range(len(track_data_json)):
    try:
        dict = merge(track_data_json[str(i)], meta_json[str(i)])
        name = {'Playlists': []}
        dict_2 = merge(dict, name)
        final[i] = dict_2
    except:
        pass

In [7]:
#ADD PLAYLIST TO EACH TRACK
for k in range(len(playlist_json)):
    name = playlist_json[k]['name']
    list_tracks = playlist_json[k]['tracks']
    for k in list_tracks:
        try: 
            final[k]['Playlists'].append(name)
        except:
            pass

In [8]:
#CONVERT MERGED DICTONARY TO DATAFRAME
tracks_df = pd.DataFrame.from_dict(final, orient='index')

In [9]:
def read_pickle(path_o, name_in):
    import pickle
    tmp_data = pickle.load(open(path_o + name_in + ".pk", "rb"))
    return tmp_data

def write_pickle(obj_in, path_o, name_in):
    import pickle
    pickle.dump(obj_in, open(path_o + name_in + ".pk", "wb"))
    return 0

In [14]:
write_pickle(tracks_df, 'pickle/', 'tracks_df')

0

In [11]:
tracks_df = read_pickle('pickle/', 'tracks_df')

In [12]:
def sent_fun(str_in):
    #COMPUTES COMPOUND POLARITY SCORE OF VADER SENTIMENT OF A STRING
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    senti = SentimentIntensityAnalyzer()
    ex = senti.polarity_scores(str_in)["compound"]
    return ex

In [13]:
tracks_df["Vader_compound"] = tracks_df.track_name.apply(sent_fun)